# AdVantage
adVantage is a collection of tools to be able to detect moving aircraft in a video from the vantage platform. Below is a walkthrough how to perform the tracking process.

https://vantage.earthi.world/explore/

### First install requirements

In [1]:
!python3 -m pip install -r requirements.txt

  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
     |████████████████████████████████| 118 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 15.9 MB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 36.7 MB/s eta 0:00:01
     |████████████████████████████████| 147.2 MB 183 kB/s  eta 0:00:01
     |████████████████████████████████| 1.2 MB 69.1 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 292 kB 31.0 MB/s eta 0:00:01
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


### Download yolov5 repository

Yolov5 repository needs to be installed before the AdVantage example will work. You will need to clone the repository into the current working directory.

In [20]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 11162, done.
remote: Total 11162 (delta 0), reused 0 (delta 0), pack-reused 11162
Receiving objects: 100% (11162/11162), 11.15 MiB | 11.21 MiB/s, done.
Resolving deltas: 100% (7716/7716), done.


In [24]:
%cd yolov5
%cd input

/Users/jw/Projects/hackathon/AdVantage/yolov5
[Errno 2] No such file or directory: 'input'
/Users/jw/Projects/hackathon/AdVantage/yolov5


### Setup yolov5 and download weights

In [21]:
%cd yolov5
!python3 -m pip install -r requirements.txt
%cd 'input'
!wget "https://doc-0g-8o-docs.googleusercontent.com/docs/securesc/qkaugo6314u8fv5lmscc33kqv5o3bopg/esvgvhjj10btnsk4g68shecqqoktphlh/1646479875000/07750003106284155945/16543569848639810509/1d0yM-NOQU8UAsfwmHIA08O_0q0zrIZLt?e=download&ax=ACxEAsYkJsl0iaJ-_j2SpzTlnNaraQ1SR8vugOQw9Z_28iMomB8qwbHjWc8MYEQVFgWRV-dkB7bqzpvCF2PZlFUwAoqIpdoaBhuxq8SuVV1XYVOJWRhmeY3obuLdMMw1v-TF1YJrdQobETTPbhccdOcF3MRw1bIfoTVkvi1HyMfFkK40miijIeszK9jmBdheS42jVIdow5AkKew-AappaIjoVyi4UFG9Qx9ooNiOCjzVvZzIEIWnH-kHhaC5gqBT7FKNdYrruMa5lJG8fjqqrhoCv_8ybjgoWqpZLLu3V1TxxEpX4JhdGs9Hn6Tond3gwuHFGBZjdrB72i_nbpDRcISzm3AKVxZoG9lKFuVufZMk_XhRPo_AijG-ZsuxoJV2swZbvqKrge0bQjf9swLq-WG4osA_hF-IJGyDlfgvEfnP8ivlbDnA7URrr9z8UiKEvhgF12OtxOARlR-pEStuz8xH97Y0qmY4L43O0VEyNvENnj64azs1t3wmxIaoA-S7D1aKWniFlqBu_7LlJ30MQ0rCoaES3Xys02bhn_rZKkRRrBzz2LMmijHkCtF2mtSIseihuTuk44JIKBD4KSPLmIZQ6Dplr9jlpOh5NJLofzKpBJeeDHQOwxEboXy1LkjUjd5YR3gjgBq9zFbOfyskjtEQMLafBTlJPMTPdGJHlXk&authuser=1&nonce=mcq8019j2dkbm&user=16543569848639810509&hash=832393uruv0rtilcoehviq5c8tm135k5"
%cd ../..

/Users/jw/Projects/hackathon/AdVantage/yolov5
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
[Errno 2] No such file or directory: 'input'
/Users/jw/Projects/hackathon/AdVantage/yolov5
The destination name is too long (826), reducing to 236
--2022-03-05 11:35:14--  https://doc-0g-8o-docs.googleusercontent.com/docs/securesc/qkaugo6314u8fv5lmscc33kqv5o3bopg/esvgvhjj10btnsk4g68shecqqoktphlh/1646479875000/07750003106284155945/16543569848639810509/1d0yM-NOQU8UAsfwmHIA08O_0q0zrIZLt?e=download&ax=ACxEAsYkJsl0iaJ-_j2SpzTlnNaraQ1SR8vugOQw9Z_28iMomB8qwbHjWc8MYEQVFgWRV-dkB7bqzpvCF2PZlFUwAoqIpdoaBhuxq8SuVV1XYVOJWRhmeY3obuLdMMw1v-TF1YJrdQobETTPbhccdOcF3MRw1bIfoTVkvi1HyMfFkK40miijIeszK9jmBdheS42jVIdow5AkKew-AappaIjoVyi4UFG9Qx9ooNiOCjzVvZzIEIWnH-kHhaC5gqBT7FKNdYrruMa5lJG8fjqqrhoCv_8ybjgoWqpZLLu3V1TxxEpX4JhdGs9Hn6Tond3gwuHFGBZjdrB72i_nbpDRcISzm3AKVxZoG9lKFuVufZMk_XhRPo_AijG-ZsuxoJV2swZbvqKrge0bQjf9swLq-WG4osA_h

### Authenticate with the Vantage platform
To interact with the api you will need an API key that can be generated on the vantage platform. If you do not have this please generate a key from your account area.
https://vantage.earthi.world/advanced/#/account . Your username will be the email address you used to sign up with

In [ ]:
import getpass
from vantage_api.services import VantageServiceApi

user = input('Vantage Email Address: ')
apiKey = getpass.getpass('API Key:')

vantageApi = VantageServiceApi(user, apiKey)

### Searching for an Airport
You are able to search for airports using the airport database. If you know the IATA code you can use the code below to find the airport. The result will give to longitude and latiute information you can use to search for videos on the Vantage platform. A couple of codes that can be used here:

* AKL

In [ ]:
from airport_database.airports import Airports
from vantage_api.utils import GeoRequest

iataCode = input('IATA Code: ')
airports = Airports()
airport = airports.findByIATACode(iataCode.upper())
print('Airport: '+airport.name)
print('Latitude: '+str(airport.lat_decimal))
print('Longitude: '+str(airport.lon_decimal))

geoData = GeoRequest()
geoData.fromBoundingBox(airport.boundingBox(halfSideInKm=10))

features = vantageApi.getSearch(geoData).getFeatures()
for key in features.keys():
    print(key)

chosenFeature = None
while chosenFeature == None:
    chosenKey = input('Enter a Key to Process: ')    
    if chosenKey in features.keys():
        chosenFeature = features[chosenKey]        

print('Chosen: '+chosenFeature)

### Download Video From Vantage Platform
Downloading the image from the vantage platform requires you to create a job and wait for its result. The API wrapper on AdVantage will allow you to do this in one command.

You can also download a stablised video using the `stabliseVideo` service.

Please choose one of two pieces of code to start the download. 

**Please note it can take quite some time to download these videos from the Vantage platform**

In [ ]:
outputFiles = vantageApi.downloadFiles(chosenFeature,'./input', verbose=True)
print(outputFiles)

In [ ]:
outputFiles = vantageApi.stabliseVideo(chosenFeature,'./input', verbose=True)
print(outputFiles)

### Detect Aircraft Movements
The AdVantage platform is written to run via a pipeline. This allows you to easily create tasks for each frame of the video and perform tasks

In [8]:
import os.path
import zipfile
import sys
sys.path.insert(0, './yolov5')
from advantage.app import AdVantage
from advantage.handlers import *

#Test file if downloads on Vantage are not working - File must be in the input folder
#chosenFeature = 'https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b9.zip'
chosenFeature = './input/stabalised_VX020001c0.mp4'

app = AdVantage()
cwd = '.'
outputDir = os.path.join(cwd, 'output')

if not os.path.exists(outputDir):
    os.makedirs(outputDir)
   
filename, file_extension = os.path.splitext(chosenFeature)
filename = os.path.basename(filename)
isZip = file_extension == '.zip'
if isZip:
    print('unzipping files')
    with zipfile.ZipFile(os.path.join(cwd,'input',filename+'.zip'), 'r') as zip_ref:
        zip_ref.extractall('input/')


pipelines = [
    #Verbose(),
    VideoWriter(outputDir+'/'+filename+'.mp4'),
    YoloProcessor(os.path.join(cwd,'input','best.pt')), 
    VideoPredictionVisualisation(),
]

if isZip:
    pipelines.append(VideoAttachGeoData(os.path.join(cwd,'input',filename+'_geometry.xml')))

outputVideoPath = os.path.join(cwd,'input', filename+'.mp4')
print('Processing Video Please Wait')
result = app.process_video(outputVideoPath, app.pipeline_factory(pipelines))
print('Video Created: '+outputVideoPath)   
app.save_output_to_json(result, os.path.join(cwd,'output','results.json'))



YOLOv5 🚀 v6.1-14-g8a66eba torch 1.10.2 CPU



FileNotFoundError: [Errno 2] No such file or directory: 'input/best.pt'

# Pipeline todo
- get video from vantage platform (or wget a pre-downloaded video for POC)
- split into frames  or load video frame per frame using open cv
- run yolo on each frame recording bbox of each image in yolo format
- convert yolo format bbox to geographic format using frame metadata and projection
    - display on map for POC
- create binary image which is geo based
- run particle filter or something similar using CV2 to track objects
- use tracked objects positioning etc. to calculate speed and heading
- display this both in a map and on the original video